In [1]:
import cifar10_input
import tensorflow as tf
import numpy as np
import os

In [2]:
#取数据
batch_size = 128
workPath = os.getcwd()
data_dir = 'tmp/cifar10_data/cifar-10-batches-bin'
dest_directory = os.path.join(workPath, data_dir)
print("begin")
images_train, labels_train = cifar10_input.inputs(eval_data = False,data_dir = dest_directory, batch_size = batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data = True, data_dir = dest_directory, batch_size = batch_size)
print("begin data")

begin
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.F

In [3]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 24,24,3]) # cifar data image of shape 24*24*3
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 数字=> 10 classes


x_image = tf.reshape(x, [-1,24,24,3])

h_conv1 =tf.contrib.layers.conv2d(x_image,64,[5,5],1,'SAME',activation_fn=tf.nn.relu)
h_pool1 = tf.contrib.layers.max_pool2d(h_conv1,[2,2],stride=2,padding='SAME')


h_conv2 =tf.contrib.layers.conv2d(h_pool1,64,[5,5],1,'SAME',activation_fn=tf.nn.relu)
h_pool2 = tf.contrib.layers.max_pool2d(h_conv2,[2,2],stride=2,padding='SAME')


nt_hpool2 = tf.contrib.layers.avg_pool2d(h_pool2,[6,6],stride=6,padding='SAME')

nt_hpool2_flat = tf.reshape(nt_hpool2, [-1, 64])

y_conv = tf.contrib.layers.fully_connected(nt_hpool2_flat,10,activation_fn=tf.nn.softmax)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
cross_entropy = -tf.reduce_sum(y*tf.log(y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [5]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tf.train.start_queue_runners(sess=sess)
for i in range(15000):#20000
    image_batch, label_batch = sess.run([images_train, labels_train])
    label_b = np.eye(10,dtype=float)[label_batch] #one hot
  
    train_step.run(feed_dict={x:image_batch, y: label_b},session=sess)
  
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:image_batch, y: label_b},session=sess)
        print( "step %d, training accuracy %g"%(i, train_accuracy))

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
step 0, training accuracy 0.117188
step 200, training accuracy 0.335938
step 400, training accuracy 0.375
step 600, training accuracy 0.367188
step 800, training accuracy 0.414062
step 1000, training accuracy 0.429688
step 1200, training accuracy 0.476562
step 1400, training accuracy 0.375
step 1600, training accuracy 0.429688
step 1800, training accuracy 0.414062
step 2000, training accuracy 0.492188
step 2200, training accuracy 0.507812
step 2400, training accuracy 0.492188
step 2600, training accuracy 0.546875
step 2800, training accuracy 0.5
step 3000, training accuracy 0.445312
step 3200, training accuracy 0.4375
step 3400, training accuracy 0.5
step 3600, training accuracy 0.53125
step 3800, training accuracy 0.359375
step 4000, training accuracy 0.515625
step 4200, training accuracy 0.515625
step 4400, training accuracy 0.59375
step 4600, training accuracy 0.570312
step 4800, training accuracy 0.5

In [6]:
image_batch, label_batch = sess.run([images_test, labels_test])
label_b = np.eye(10,dtype=float)[label_batch]#one hot
print ("finished！ test accuracy %g"%accuracy.eval(feed_dict={
     x:image_batch, y: label_b},session=sess))

finished！ test accuracy 0.65625
